In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from decimal import *
TWOPLACES = Decimal(10) ** -2
from natsort import natsorted
import ast
import time
today = time.strftime("%Y_%m_%d")
from enrich_dimensions.rounds import rounds, rounding_inch_feet,rounding_lbs,rounding_w,rounding_oz, rounding_lb,rounding_gal, re_extract, curate, round_string_float, clean_data,reg_ex,rounding_period_after_unit 
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A

def filterz(df,pat):
    df['m_long']=df['long_desc'].apply(lambda x: re_extract(pat,str(x)))
    df['m_name']=df['product_name'].apply(lambda x: re_extract(pat,str(x)))
    df['m_custom']=df['custom_fields'].apply(lambda x: re_extract(pat,str(x)))
    return df

In [2]:
# parameters
customer_id = '143'
formatted_attribute = 'Gender'
customer_name='%liverpool%'
buckets = """Office Chairs"""

attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'


df = query_from_file(file_name='./query/custom_fields.sql', params=params)
print(len(df))
print(today)

3075
2023_03_31


In [4]:
pat='''(?i)(.{0,30}(?:gender|\bman\b|woman|boy|girl|\bmen\b|women|unisex|for women|female|(?<!fe)male|lady).{0,30})|()'''
# pat='''(?i)(.{0,30}(?:gender\s?(?:man|woman|women|girl|boy|unisex)|for\s?(?:man|woman|women|girl|boy|unisex)).{0,10})|()'''


df['m_longs']=df['long_desc'].apply(lambda x: re_extract(pat,str(x)))
df['m_names']=df['product_name'].apply(lambda x: re_extract(pat,str(x)))
df['m_customs']=df['custom_fields'].apply(lambda x: re_extract(pat,str(x)))
gender=df.loc[(df['m_longs'].astype(str)!='[]')|(df['m_names'].astype(str)!='[]')|(df['m_customs'].astype(str)!='[]')]
na=df.loc[(df['m_longs'].astype(str)=='[]')&(df['m_names'].astype(str)=='[]')&(df['m_customs'].astype(str)=='[]')]
print(len(gender))
print(len(na))

1699
1376


In [5]:
print(len(na))

1376


In [6]:
pat='''(?i)((?:\bman\b|woman|boy|girl|\bmen\b|women|unisex|for women|female|(?<!fe)male|lady))|()'''
gen=filterz(gender,pat)
print(len(gen))

1699


In [7]:
long=gen[(gen['m_longs'].astype(str)!='[]')&(gen['m_longs'].astype(str)!='Gender')]
print(len(long))

1565


In [59]:
long['Q:gender']=long['m_longs'].apply(lambda x: re.sub(r'\s?for\s?','',str(x))).apply(lambda x: re.sub(r'((?:woman|female|women|lady|WOMAN))','Woman',str(x))).apply(lambda x: x.title()).apply(lambda x: re.sub(r"((?:Lady|Female))",'Woman',str(x))).apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)]))#.apply(lambda x: natsorted(x)).apply(lambda x: re.sub(r'''(((?:"|')?\s?\,\s?(?!")(?:')?))''','","',str(x))).apply(lambda x: re.sub(r'''((?<=\[)\s?'\s?)|(\s?'\s?(?=\]))|(\s?'\s?(?=\,))|((?<=\,)\s?'\s?)''','"',str(x)))             
long['Q:gender']=long['Q:gender'].apply(lambda x: re.sub(r'(?i)(.+Gender\s?)','["',str(x))).apply(lambda x: re.sub(r'(?i)((?<=Man).+)|((?<=Unisex).+)','"]',str(x)))

In [60]:
long['Q:gender'].explode().value_counts()[0:500]

["Man"]                               1145
["Unisex"]                             346
["Child Ip Protection Degree No']        6
["Child Calendar Does Not Have']         4
["Child Includes No Additional']         4
["Child Watch Crystal Acrylic S']        4
["Children Unisex"]                      4
["Brown Man"]                            2
["Child Watch Glass Acrylic Sha']        2
["Child Water Resistance Not Wa']        2
["Child Man"]                            2
[". Men'S Brand Skechers. Batter"]       1
["Child Family Or Collection Wi']        1
["Woman"]                                1
["Men'S Chronograph Yes Stopwat"]        1
["Child Operation Digital Man"]          1
["Child Dial No Numbers Ip Prot']        1
["Child Dial Arabic Numerals Wa']        1
["Girl Brand That'S It Water Re"]        1
["Child Chronograph No Ip Prote']        1
["Men Band Material Leather Wat']        1
["Men Water Resistance 100 M']           1
["Men Brand Boss']                       1
["Child Chr

In [67]:
man_uni=long[(long['Q:gender'].astype(str)=='["Man"]')|(long['Q:gender'].astype(str)=='["Unisex"]')]

In [68]:
match_man_uni=man_uni[['external_id','Q:gender']]

In [23]:
pats='''(?i)((?:unisex|boys and girls|girls and boys|men and women|women and men))'''
gen_no_unisex=filterz(gen,pats)

In [24]:
not_unisex=gen_no_unisex[(gen_no_unisex['m_long'].astype(str)=='[]')&(gen_no_unisex['m_name'].astype(str)=='[]')&(gen_no_unisex['m_custom'].astype(str)=='[]')]

In [25]:
print(len(not_unisex))

1344


In [26]:
pat='''(?i)((?:\bman\b|woman|boy(?!friend)|girl(?!friend)(?! brand)|\bmen\b|women|unisex|for women|female|(?<!fe)male|lady))|()'''
a=filterz(not_unisex,pat)

In [27]:
print(len(a))

1344


In [28]:
a['Q:gender']=a['m_long'].apply(lambda x: re.sub(r'\s?for\s?','',str(x))).apply(lambda x: re.sub(r'(?i)((?:woman|female|women|lady|WOMAN|Women|lady|female))','Woman',str(x))).apply(lambda x: re.sub(r"'",'"',str(x))).apply(lambda x: x.title()).apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)])).apply(lambda x: natsorted(x)).apply(lambda x: re.sub(r'''(((?:"|')?\s?\,\s?(?!")(?:')?))''','","',str(x))).apply(lambda x: re.sub(r'''((?<=\[)\s?'\s?)|(\s?'\s?(?=\]))|(\s?'\s?(?=\,))|((?<=\,)\s?'\s?)''','"',str(x)))              


In [29]:
a['Q:gender'].explode().value_counts()[0:500]

[]    1344
Name: Q:gender, dtype: int64

In [30]:
b=a[a['Q:gender'].astype(str)!='[]']

In [31]:
print(len(b))

0


In [32]:
b['Q:gender']=b['Q:gender'].apply(lambda x: re.sub(r'(\["Girl","Woman"\])|(\["Woman","Woman"\])','Woman',str(x))).apply(lambda x: re.sub(r'Male','Men',str(x))).apply(lambda x: re.sub(r'Woman','Women',str(x))).apply(lambda x: re.sub(r'Boys?','Boys',str(x))).apply(lambda x: re.sub(r'Girls?','Girls',str(x)))

In [33]:
b['Q:gender'].explode().value_counts()[0:500]

Series([], Name: Q:gender, dtype: int64)

In [137]:
b[b['Q:gender'].astype(str)=='["Girl","Woman"]']

Empty DataFrame
Columns: [buckets, bucket_id, product_id, external_id, product_name, long_desc, url, image_url, custom_fields, value, m_long, m_name, m_custom, m_longs, m_names, m_customs, Q:gender]
Index: []

In [138]:
match_gender=b[['external_id','Q:gender']]

In [2]:
stop

NameError: name 'stop' is not defined

In [139]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/Liverpool - {attribute}-{get_df_name(matches)}-{today}-match_gender.csv',index=False) 
looks_good('Liverpool', attribute,match_gender,today)   

In [69]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/Liverpool - {attribute}-{get_df_name(matches)}-{today}-match_man_uni.csv',index=False) 
looks_good('Liverpool', attribute,match_man_uni,today)   

In [70]:
# parameters
customer_id = '143'
formatted_attribute = 'Water Resistance'
customer_name='%liverpool%'
buckets = """Office Chairs"""

attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'


water = query_from_file(file_name='./query/custom_fields.sql', params=params)
print(len(water))
print(today)

197
2023_03_31


In [4]:
# anything that calls out water proof
pat='''(?i)(.{0,30}(?:water.?proof|water.?resistance|resistance.?into.?water|resistance|resistance.?water|resistance.?to.?water).{0,20})|()'''
water['m_longs']=water['long_desc'].apply(lambda x: re_extract(pat,str(x)))
water['m_names']=water['product_name'].apply(lambda x: re_extract(pat,str(x)))
water['m_customs']=water['custom_fields'].apply(lambda x: re_extract(pat,str(x)))


# filter out NOT water proof
pat='''(?i)(not water.?proof)|()'''
water['na_long']=water['m_longs'].apply(lambda x: re_extract(pat,str(x)))
water['na_name']=water['m_names'].apply(lambda x: re_extract(pat,str(x)))
water['na_custom']=water['m_customs'].apply(lambda x: re_extract(pat,str(x)))


# Find actual number 
pats='''(?i)(\d+\s?m(?:eters?)?\b[^\d]{0,30}(?:water.?proof|water.?resistance|resistance.?into.?water|resistance|resistance.?water|resistance.?to.?water))|((?:water.?proof|water.?resistance|resistance.?into.?water|resistance|resistance.?water|resistance.?to.?water)[^\d]{0,30}\d+\s?m(?:eters?)?\b)|()'''                 
water['m_l']=water['m_longs'].apply(lambda x: re_extract(pat,str(x)))
water['m_n']=water['m_names'].apply(lambda x: re_extract(pat,str(x)))
water['m_c']=water['m_customs'].apply(lambda x: re_extract(pat,str(x)))



resistance=water.loc[((water['m_longs'].astype(str)!='[]')|(water['m_names'].astype(str)!='[]')|(water['m_customs'].astype(str)!='[]'))&((water['na_long'].astype(str)=='[]')&(water['na_name'].astype(str)=='[]')&(water['na_custom'].astype(str)=='[]'))]
na_water=water.loc[(water['m_longs'].astype(str)=='[]')&(water['m_names'].astype(str)=='[]')&(water['m_customs'].astype(str)=='[]')]
print(len(resistance))
print(len(na_water))

3153
28


# NA

In [5]:
# DOES NOT CALL OUT WATER PROOF
na_wat=water[(water['na_long'].astype(str)!='[]')|(water['na_name'].astype(str)!='[]')|(water['na_custom'].astype(str)!='[]')]
print(len(na_wat))

93


In [6]:
# Calls out NOT Water Proof
na_water=water[(water['m_longs'].astype(str)=='[]')&(water['m_names'].astype(str)=='[]')&(water['m_customs'].astype(str)=='[]')]
print(len(na_water))

28


# Curation

In [7]:
print(len(resistance))

3153


In [9]:
pat='''(?i)(\d+\s?m)|()'''
resistance['Q:water_resistance']=resistance['m_longs'].apply(lambda x: re_extract(pat,str(x)))

In [51]:
resistance['Q:water_resistance']=resistance['Q:water_resistance'].apply(lambda x: re.sub(r'(?i)((?<=\d)m)',' m',str(x))).apply(lambda x: re.sub(r"\s?'\s?",'"',str(x))).apply(lambda x: re.sub(r'\s"','"',str(x))).apply(lambda x: re.sub(r'(?i)(\s?m)',' m',str(x)))       
# resistance['Q:water_resistance'].explode().value_counts()

In [11]:
r=resistance[resistance['Q:water_resistance'].astype(str)!='[]']
pattern='''(?i)((?<!\d)(?:1000 m|100 m|10 m|200 m|30 m|50 m|5 m|66 m))|()'''
r['junk']=r['Q:water_resistance'].apply(lambda x: re_extract(pattern,str(x)))

In [109]:
pats='''(?i)(\d+\s?m(?:eters?)?(?!m)[^\d]{0,5}(?:water.?proof|water.?resistance|resistance.?into.?water|resistance|resistance.?water|resistance.?to.?water|water.?(?:resistance|proof).?(?:up.?to|to)?)(?! \d)(?!\d))|((?<!\d m )(?<!\dm )(?<!\dm)(?<!\d m)(?:water.?proof|water.?resistance|resistance.?into.?water|resistance|resistance.?water|resistance.?to.?water|water.?(?:resistance|proof)\s?(?:up.?to|to|into))[^\d]{0,5}\d+\s?m(?:eters?)?)|()'''                 
resistance['m_l']=resistance['m_longs'].apply(lambda x: re_extract(pats,str(x)))
resistance['m_n']=resistance['m_names'].apply(lambda x: re_extract(pats,str(x)))
resistance['m_c']=resistance['m_customs'].apply(lambda x: re_extract(pats,str(x)))

In [110]:
resist=resistance[(resistance['m_l'].astype(str)!='[]')|(resistance['m_n'].astype(str)!='[]')|(resistance['m_c'].astype(str)!='[]')]
print(len(resist))

3082


In [111]:
m_l=resist[resist['m_l'].astype(str)!='[]']
print(len(m_l))

3082


In [126]:
m_l['Q:water_resistance']=m_l['m_l'].apply(lambda x: re.sub(r"'",'"',str(x))).apply(lambda x: re.sub(r'(?i)(\s?m(?:eters?)?)',' m',str(x))).apply(lambda x: re.sub(r'\s"','"',str(x))).apply(lambda x: re.sub(r'(?i)(\s?(?:resistance.?to.?water.?at|water.?resistance\s?(?:up.?to\s?|\s?of|\s?to|\s?at)?\s?|resistance.?to.?water|resistance.?(?:into|to)\s?water(?:\s?at)?|resistance water|water.?proof(?:\s?of|\sto)?|resistance(?= \d))\s?)|([a-ln-z])|()','',str(x))).apply(lambda x: re.sub(r'\:\s','',str(x))).apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)])).apply(lambda x: natsorted(x)).apply(lambda x: re.sub(r'''(((?:"|')?\s?\,\s?(?!")(?:')?))''','","',str(x))).apply(lambda x: re.sub(r'''((?<=\[)\s?'\s?)|(\s?'\s?(?=\]))|(\s?'\s?(?=\,))|((?<=\,)\s?'\s?)''','"',str(x)))                   


In [127]:
m_l[0:500]
# m_l[m_l['Q:water_resistance'].astype(str)=='["6 months Shock resistance","50 m"]']
m_l['Q:water_resistance'].explode().value_counts()

["30 m"]            1334
["50 m"]            1098
["100 m"]            321
["10 m"]             153
["200 m"]            111
["5 m"]               47
["1000 m"]             7
["20 m"]               3
["30 m","50 m"]        2
["300 m"]              2
["50 m","200 m"]       2
["50 m","100 m"]       1
["30 m","100 m"]       1
Name: Q:water_resistance, dtype: int64

In [128]:
m=m_l[(m_l['Q:water_resistance'].astype(str)!='["300 m"]')&(m_l['Q:water_resistance'].astype(str)!='["20 m"]')]
print(len(m))

mm=m_l[(m_l['Q:water_resistance'].astype(str)=='["300 m"]')|(m_l['Q:water_resistance'].astype(str)=='["20 m"]')]
print(len(mm))

3077
5


In [130]:
match_water_resistance=m[['external_id','Q:water_resistance']]

stop

In [131]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/Liverpool - {attribute}-{get_df_name(matches)}-{today}-match_water_resistance.csv',index=False) 
looks_good('Liverpool', attribute,match_water_resistance,today)   

# Material

In [71]:
# parameters
customer_id = '143'
formatted_attribute = 'Material'
customer_name='%liverpool%'
buckets = """Office Chairs"""

attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'


material = query_from_file(file_name='./query/custom_fields.sql', params=params)
print(len(material))
print(today)

1170
2023_03_31


In [136]:
def curation(water,pat, pats, patz):
    # anything that calls out water proof
#     pat='''(?i)(.{0,30}(?:water.?proof|water.?resistance|resistance.?into.?water|resistance|resistance.?water|resistance.?to.?water).{0,20})|()'''
    water['m_longs']=water['long_desc'].apply(lambda x: re_extract(pat,str(x)))
    water['m_names']=water['product_name'].apply(lambda x: re_extract(pat,str(x)))
    water['m_customs']=water['custom_fields'].apply(lambda x: re_extract(pat,str(x)))


    # filter out NOT water proof
#     pats='''(?i)(not water.?proof)|()'''
    water['na_long']=water['m_longs'].apply(lambda x: re_extract(pats,str(x)))
    water['na_name']=water['m_names'].apply(lambda x: re_extract(pats,str(x)))
    water['na_custom']=water['m_customs'].apply(lambda x: re_extract(pats,str(x)))


    # Find actual number 
#     patz='''(?i)(\d+\s?m(?:eters?)?\b[^\d]{0,30}(?:water.?proof|water.?resistance|resistance.?into.?water|resistance|resistance.?water|resistance.?to.?water))|((?:water.?proof|water.?resistance|resistance.?into.?water|resistance|resistance.?water|resistance.?to.?water)[^\d]{0,30}\d+\s?m(?:eters?)?\b)|()'''                 
    water['m_l']=water['m_longs'].apply(lambda x: re_extract(patz,str(x)))
    water['m_n']=water['m_names'].apply(lambda x: re_extract(patz,str(x)))
    water['m_c']=water['m_customs'].apply(lambda x: re_extract(patz,str(x)))



    resistance=water.loc[((water['m_longs'].astype(str)!='[]')|(water['m_names'].astype(str)!='[]')|(water['m_customs'].astype(str)!='[]'))&((water['na_long'].astype(str)=='[]')&(water['na_name'].astype(str)=='[]')&(water['na_custom'].astype(str)=='[]'))]
    na_water=water.loc[(water['m_longs'].astype(str)=='[]')&(water['m_names'].astype(str)=='[]')&(water['m_customs'].astype(str)=='[]')]
    print(len(resistance))
    print(len(na_water))
    return resistance,na_water

In [155]:
pat='''(?i)(.{0,30}(?:material|Material).{0,30})|()'''
pats='''(?i)(abc(?:cotton|fabric|textile|faux.?leather|Curpiel|foam|leather|memory.?foam|nylon|plastic|polyester|polyurethane|recycled.?material|rubber|satin|silk|suede|synthetic|\btpr\b|Thermoplastic.?Rubber|vinyl|Vinipeil))|()'''
patz='''(?i)((?:cotton|fabric|textile|faux.?leather|Curpiel|foam|leather|memory.?foam|nylon|plastic|polyester|polyurethane|recycled.?material|rubber|satin|silk|suede|synthetic|\btpr\b|Thermoplastic.?Rubber|vinyl|Vinipeil))|()'''

mat, na_mat=curation(material,pat, pats, patz)

73
4302


In [159]:
pat='''(?i)(.{0,30}(?:material|Material).{0,30})|()'''
material['m_longs']=material['long_desc'].apply(lambda x: re_extract(pat,str(x)))
material['m_names']=material['product_name'].apply(lambda x: re_extract(pat,str(x)))
material['m_customs']=material['custom_fields'].apply(lambda x: re_extract(pat,str(x)))

In [160]:
patz='''(?i)((?:cotton|fabric|textile|faux.?leather|Curpiel|foam|leather|memory.?foam|nylon|plastic|polyester|polyurethane|recycled.?material|rubber|satin|silk|suede|synthetic|\btpr\b|Thermoplastic.?Rubber|vinyl|Vinipeil))|()'''
material['m_l']=material['m_longs'].apply(lambda x: re_extract(patz,str(x)))
material['m_n']=material['m_names'].apply(lambda x: re_extract(patz,str(x)))
material['m_c']=material['m_customs'].apply(lambda x: re_extract(patz,str(x)))

In [168]:
mat=material[(material['m_l'].astype(str)!='[]')|(material['m_n'].astype(str)!='[]')|(material['m_c'].astype(str)!='[]')]
print(len(mat))
na_mat=material[(material['m_l'].astype(str)=='[]')&(material['m_n'].astype(str)=='[]')&(material['m_c'].astype(str)=='[]')]
print(len(na_mat))

3904
471


In [172]:
m_l=mat[mat['m_l'].astype(str)!='[]']
print(len(m_l))

3896


In [180]:
m_l['Q:material']=m_l['m_l'].apply(lambda x: re.sub(r"'",'"',str(x))).apply(lambda x: x.title()).apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)])).apply(lambda x: natsorted(x)).apply(lambda x: re.sub(r'''(((?:"|')?\s?\,\s?(?!")(?:')?))''','","',str(x))).apply(lambda x: re.sub(r'''((?<=\[)\s?'\s?)|(\s?'\s?(?=\]))|(\s?'\s?(?=\,))|((?<=\,)\s?'\s?)''','"',str(x)))


In [184]:
hand=m_l[m_l['buckets'].astype(str)=='Handbags/Shoulder Bags']
every=m_l[m_l['buckets'].astype(str)=='Everyday/Dress Footwear']
back=m_l[m_l['buckets'].astype(str)=='Backpacks/Daypacks']
clutch=m_l[m_l['buckets'].astype(str)=='Clutches & Wristlets']
active=m_l[m_l['buckets'].astype(str)=='Active/Athletic Footwear']

#### Handbags

In [185]:
patz='''(?i)((?:cotton|fabric|textile|faux.?leather|Curpiel|leather|nylon|plastic|polyester|polyurethane|recycled.?material|satin|silk|synthetic|vinyl|Vinipeil))|()'''
hand['m_l']=hand['m_longs'].apply(lambda x: re_extract(patz,str(x)))
hand['m_n']=hand['m_names'].apply(lambda x: re_extract(patz,str(x)))
hand['m_c']=hand['m_customs'].apply(lambda x: re_extract(patz,str(x)))

In [190]:
print(len(hand))
m=hand[hand['m_c'].astype(str)!='[]']
print(len(m))

2402
2145


In [191]:
m['Q:material']=m['Q:material'].apply(lambda x: re.sub(r'Textile','Fabric',str(x))).apply(lambda x: re.sub(r'Curpiel','Faux Leather',str(x)))

In [192]:
m['Q:material'].explode().value_counts()

["Synthetic"]                        1656
["Polyester"]                         207
["Polyurethane"]                      109
["Fabric"]                             41
["Nylon"]                              35
["Cotton"]                             33
["Vinyl"]                              12
["Synthetic","Fabric"]                 12
["Leather"]                            11
["Leather","Synthetic"]                 8
["Polyester","Fabric"]                  5
["Synthetic","Vinyl"]                   5
["Silk"]                                5
["Plastic"]                             4
["Suede","Synthetic"]                   1
["Polyester","Recycled Material"]       1
Name: Q:material, dtype: int64

In [193]:
match_hand=m[['external_id','Q:material']]

#### Everyday

In [195]:
patz='''(?i)((?:fabric|textile|faux.?leather|Curpiel|foam|leather|memory.?foam|nylon|polyurethane|recycled.?material|rubber|suede|synthetic|\btpr\b|Thermoplastic.?Rubber|vinyl|Vinipeil))|()'''
every['m_l']=every['m_longs'].apply(lambda x: re_extract(patz,str(x)))
every['m_n']=every['m_names'].apply(lambda x: re_extract(patz,str(x)))
every['m_c']=every['m_customs'].apply(lambda x: re_extract(patz,str(x)))

In [196]:
print(len(every))
m=every[every['m_c'].astype(str)!='[]']
print(len(m))

1042
775


In [197]:
m['Q:material']=m['Q:material'].apply(lambda x: re.sub(r'Textile','Fabric',str(x))).apply(lambda x: re.sub(r'Curpiel','Faux Leather',str(x)))

In [198]:
m['Q:material'].explode().value_counts()

["Synthetic"]             532
["Fabric"]                146
["Suede"]                  80
["Polyurethane"]            8
["Leather"]                 6
["Synthetic","Fabric"]      3
Name: Q:material, dtype: int64

In [199]:
match_every=m[['external_id','Q:material']]

#### Backpack

In [201]:
patz='''(?i)((?:cotton|fabric|textile|faux.?leather|Curpiel|leather|nylon|plastic|polyester|polyurethane|satin|silk|synthetic|vinyl|Vinipeil))|()'''
back['m_l']=back['m_longs'].apply(lambda x: re_extract(patz,str(x)))
back['m_n']=back['m_names'].apply(lambda x: re_extract(patz,str(x)))
back['m_c']=back['m_customs'].apply(lambda x: re_extract(patz,str(x)))

In [202]:
print(len(back))
m=back[back['m_c'].astype(str)!='[]']
print(len(m))

289
267


In [203]:
m['Q:material']=m['Q:material'].apply(lambda x: re.sub(r'Textile','Fabric',str(x))).apply(lambda x: re.sub(r'Curpiel','Faux Leather',str(x)))

In [204]:
m['Q:material'].explode().value_counts()

["Synthetic"]                164
["Polyester"]                 37
["Nylon"]                     19
["Fabric"]                    17
["Polyurethane"]              10
["Cotton"]                     5
["Leather","Synthetic"]        3
["Polyester","Fabric"]         3
["Polyester","Synthetic"]      3
["Satin"]                      2
["Vinyl"]                      2
["Plastic"]                    2
Name: Q:material, dtype: int64

In [205]:
match_back=m[['external_id','Q:material']]

In [ ]:
mat[0:500]
# m_l[m_l['Q:water_resistance'].astype(str)=='["6 months Shock resistance","50 m"]']
mat['Q:water_resistance'].explode().value_counts()

In [ ]:
match_material=matttt[['external_id','Q:material']]

In [ ]:
stop

In [194]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/Liverpool - {attribute}-{get_df_name(matches)}-{today}-match_hand.csv',index=False) 
looks_good('Liverpool', attribute,match_hand,today)   

In [200]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/Liverpool - {attribute}-{get_df_name(matches)}-{today}-match_every.csv',index=False) 
looks_good('Liverpool', attribute,match_every,today)  

In [206]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/Liverpool - {attribute}-{get_df_name(matches)}-{today}-match_back.csv',index=False) 
looks_good('Liverpool', attribute,match_back,today) 

# Ram

In [74]:
# parameters
customer_id = '143'
formatted_attribute = 'Random Access Memory RAM'
customer_name='%liverpool%'
buckets = """Office Chairs"""

attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'


ram = query_from_file(file_name='./query/custom_fields.sql', params=params)
print(len(ram))
print(today)

200
2023_03_31


In [76]:
def curation(water,pat, pats, patz):
    # anything that calls out water proof
#     pat='''(?i)(.{0,30}(?:water.?proof|water.?resistance|resistance.?into.?water|resistance|resistance.?water|resistance.?to.?water).{0,20})|()'''
    water['m_longs']=water['long_desc'].apply(lambda x: re_extract(pat,str(x)))
    water['m_names']=water['product_name'].apply(lambda x: re_extract(pat,str(x)))
    water['m_customs']=water['custom_fields'].apply(lambda x: re_extract(pat,str(x)))


    # filter out NOT water proof
#     pats='''(?i)(not water.?proof)|()'''
    water['na_long']=water['m_longs'].apply(lambda x: re_extract(pats,str(x)))
    water['na_name']=water['m_names'].apply(lambda x: re_extract(pats,str(x)))
    water['na_custom']=water['m_customs'].apply(lambda x: re_extract(pats,str(x)))


    # Find actual number 
#     patz='''(?i)(\d+\s?m(?:eters?)?\b[^\d]{0,30}(?:water.?proof|water.?resistance|resistance.?into.?water|resistance|resistance.?water|resistance.?to.?water))|((?:water.?proof|water.?resistance|resistance.?into.?water|resistance|resistance.?water|resistance.?to.?water)[^\d]{0,30}\d+\s?m(?:eters?)?\b)|()'''                 
    water['m_l']=water['m_longs'].apply(lambda x: re_extract(patz,str(x)))
    water['m_n']=water['m_names'].apply(lambda x: re_extract(patz,str(x)))
    water['m_c']=water['m_customs'].apply(lambda x: re_extract(patz,str(x)))



    resistance=water.loc[((water['m_longs'].astype(str)!='[]')|(water['m_names'].astype(str)!='[]')|(water['m_customs'].astype(str)!='[]'))&((water['na_long'].astype(str)=='[]')&(water['na_name'].astype(str)=='[]')&(water['na_custom'].astype(str)=='[]'))]
    na_water=water.loc[(water['m_longs'].astype(str)=='[]')&(water['m_names'].astype(str)=='[]')&(water['m_customs'].astype(str)=='[]')]
    print(len(resistance))
    print(len(na_water))
    return resistance,na_water

In [95]:
pat='''(?i)(.{0,30}\d+\s?GB.{0,30})|()'''
pats='''(?i)(gizzz)|()'''
patz='''(?i)(Ram memory capacity (?:12|16|32|4|8) GB)|()'''

rams, na_ram=curation(ram, pat, pats, patz)

200
0


In [96]:
print(len(rams))


200


In [97]:
r=rams[rams['m_l'].astype(str)!='[]']
print(len(r))

177


In [102]:
r[curation_col]=r['m_l'].apply(lambda x: re.sub(r'(?i)(\s?GB)',' GB',str(x))).apply(lambda x: re.sub(r'(?i)(Ram Memory Capacity )','',str(x))).apply(lambda x: re.sub(r"\s?'\s?",'"',str(x))).apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)])).apply(lambda x: natsorted(x)).apply(lambda x: re.sub(r'''(((?:"|')?\s?\,\s?(?!")(?:')?))''','","',str(x))).apply(lambda x: re.sub(r'''((?<=\[)\s?'\s?)|(\s?'\s?(?=\]))|(\s?'\s?(?=\,))|((?<=\,)\s?'\s?)''','"',str(x)))      


In [105]:
r[curation_col].explode().value_counts()

["8 GB"]     117
["16 GB"]     41
["4 GB"]       9
["32 GB"]      7
["12 GB"]      3
Name: Q:random_access_memory_ram, dtype: int64

In [106]:
r['buckets'].explode().value_counts()

Portable Computers    177
Name: buckets, dtype: int64

In [108]:
match_ram=r[['external_id',curation_col]]

In [ ]:
stop

In [179]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/Liverpool - -{get_df_name(matches)}-{today}-match_men.csv',index=False) 
looks_good('Liverpool', match_men,today)

# Curated 

In [14]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from decimal import *
TWOPLACES = Decimal(10) ** -2
from natsort import natsorted
import ast
import time
today = time.strftime("%Y_%m_%d")
from enrich_dimensions.rounds import rounds, rounding_inch_feet,rounding_lbs,rounding_w,rounding_oz, rounding_lb,rounding_gal, re_extract, curate, round_string_float, clean_data,reg_ex,rounding_period_after_unit 
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A



def curation(df, pat):
    df['m_long']=df['long_desc'].apply(lambda x: re_extract(pat,str(x)))
    df['m_name']=df['product_name'].apply(lambda x: re_extract(pat,str(x)))
    df['m_custom']=df['custom_fields'].apply(lambda x: re_extract(pat,str(x)))
    na=df[(df['m_long'].astype(str)=='[]')&(df['m_name'].astype(str)=='[]')&(df['m_custom'].astype(str)=='[]')]
    match=df[(df['m_long'].astype(str)!='[]')|(df['m_name'].astype(str)!='[]')|(df['m_custom'].astype(str)!='[]')]
    labeled_na=df[(df['value'].astype(str)=='n/a')|(df['value'].astype(str)=='["n/a"]')]
    
    custom=df[df['m_custom'].astype(str)!='[]']
    not_custom=df[(df['m_custom'].astype(str)=='[]')&((df['m_long'].astype(str)!='[]')|(df['m_name'].astype(str)!='[]'))]
    
    print(f'Total DF: {len(df)}')
    print(f'Total number of SKUs labeled N/A: {len(labeled_na)}')
    print(f'None Found: {len(na)}')
    print(f'Matches: {len(match)}')
    print(f'Custom Fields Only: {len(custom)}')
    print(f'Not in Custom Fields: {len(not_custom)}')

    print('##########################################################')
    print('')
    return df, match,na,custom, not_custom

In [15]:
customer_id = '143'
customer_name='%liverpool%'
dateszs='2001-10-01'
dateszsz='2023-10-01'
params = {'customer_id': customer_id,
          'customer_name':customer_name,
         'dateszs':dateszs,
         'dateszsz':dateszsz}
print('start')
# dfs = query_from_file(file_name='../query/curated_all_attributes_date.sql', params=params)
df = query_from_file(file_name='query/curated_all_attributes_date_dates.sql', params=params)
print(len(df))

start
119318


In [16]:
del df['bucket_id']
del df['customer_name']
del df['curated_date']
del df['resolution']
del df['curation_tasks.curated_by']

In [17]:
pat='''(?i)(glass)|()'''

watch=df[df['attribute'].astype(str)=='watch_case_material']
watch['m_val']=watch['value'].apply(lambda x: re_extract(pat,str(x)))

In [20]:
wat=watch[watch['m_val'].astype(str)!='[]']
print(len(wat))

988


In [26]:
wat['Q:watch_case_material']=wat['value'].apply(lambda x: re.sub(r'("Glass"\,)','',str(x))).apply(lambda x: re.sub(r'(\,"Glass")','',str(x))).apply(lambda x: re.sub(r'(Glass)','n/a',str(x)))

In [27]:
wat['Q:watch_case_material'].explode().value_counts()

["Stainless Steel"]                         515
["Steel"]                                   187
n/a                                         142
["Plastic"]                                  53
["Polycarbonate"]                            39
["Metal"]                                    23
["Resin"]                                    11
["Polyurethane"]                              5
["Nylon"]                                     4
["Silver"]                                    3
["Ceramic"]                                   2
["Plastic","Stainless Steel"]                 1
["Polycarbonate","Steel"]                     1
["Nylon","Polycarbonate","Polyurethane"]      1
["Plastic","Resin"]                           1
Name: Q:watch_case_material, dtype: int64

In [29]:
match_watch_case_material=wat[['external_id','Q:watch_case_material']]

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/Liverpool - -{get_df_name(matches)}-{today}-match_watch_case_material.csv',index=False) 
looks_good('Liverpool', match_watch_case_material,today)

In [4]:
pat='''(?i)(in)|()'''


height=df[df['attribute'].astype(str)=='height']
height['m_val']=height['value'].apply(lambda x: re_extract(pat,str(x)))
width=df[df['attribute'].astype(str)=='width']
width['m_val']=width['value'].apply(lambda x: re_extract(pat,str(x)))
depth=df[df['attribute'].astype(str)=='depth']
depth['m_val']=depth['value'].apply(lambda x: re_extract(pat,str(x)))

In [5]:
h=height[height['m_val'].astype(str)!='[]']
print(len(h))
w=width[width['m_val'].astype(str)!='[]']
print(len(w))
d=depth[depth['m_val'].astype(str)!='[]']
print(len(d))

0
0
0


In [78]:
pat='''(?i)((?:depth of (?:the )?product|product depth)\s?\d+(?:\.\d+)?(?: cm)?)|()'''
d['matches']=d['long_desc'].apply(lambda x: re_extract(pat,str(x)))
dep=d[d['matches'].astype(str)!='[]']
print(len(dep))

dep_na=d[d['matches'].astype(str)=='[]']
print(len(dep_na))

61
1


In [86]:
dep['mat']=dep['matches'].apply(lambda x: re.sub(r'(?i)((?:Depth of (?:the )?product|product\s?depth\s?))','',str(x))).apply(lambda x: re.sub(r"\s?'\s?(?=\,)|(?<=\[)\s?'\s?|(?<=\,)\s?'\s?|\s?'\s?(?=\])",'"',str(x))).apply(lambda x: re.sub(r'((?<=\d)cm)|((?<=\d)(?="))',' cm',str(x)))                     


In [87]:
dep['mat'].explode().value_counts()

["10 cm"]       4
["32 cm"]       4
["5 cm"]        4
["23 cm"]       3
["27 cm"]       3
["20 cm"]       3
["17 cm"]       3
["26 cm"]       3
["25 cm"]       2
["30 cm"]       2
["21 cm"]       2
["31 cm"]       2
["8 cm"]        2
["15 cm"]       2
["12 cm"]       2
["6 cm"]        2
["14.61 cm"]    1
["24 cm"]       1
["11 cm"]       1
["36 cm"]       1
["22 cm"]       1
["19 cm"]       1
["29.5 cm"]     1
["39 cm"]       1
["2 cm"]        1
["17.5 cm"]     1
["1 cm"]        1
["35 cm"]       1
["7 cm"]        1
["16 cm"]       1
["3 cm"]        1
["32.5 cm"]     1
["4 cm"]        1
["45 cm"]       1
Name: mat, dtype: int64

In [90]:
dep['Q:depth']=dep['mat']
match_depth=dep[['external_id','Q:depth']]

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/Liverpool - -{get_df_name(matches)}-{today}-match_depth.csv',index=False) 
looks_good('Liverpool', match_depth,today)

In [97]:
pat='''(?i)((?:(?<!product )width (?!of)|width.?product.?|width of (?:the )?product(?:.?backpack\:\s?|.?bag\:\s?|.?tote\:\s?)?|product.?width(?:.?bag\:\s?|tote\:\s?)?)\s?\d+(?:\.\d+)? (?:cm)?)|()'''
w['matches']=w['long_desc'].apply(lambda x: re_extract(pat,str(x)))
wid=w[w['matches'].astype(str)!='[]']
print(len(wid))


wid['mat']=wid['matches'].apply(lambda x: re.sub(r'(?i)(Width of the product\s?)|(\s?Bag\s?\:)|(Product\s?width\s?)|(width\s?product\s?)|(width of product\s?)|(backpack\:)|(tote\:\s?)(Tote\:\s?)','',str(x))).apply(lambda x: re.sub(r"\s?'\s?(?=\,)|(?<=\[)\s?'\s?|(?<=\,)\s?'\s?|\s?'\s?(?=\])",'"',str(x))).apply(lambda x: re.sub(r'(?:)(\s?Width\s?)|(Tote\: )','',str(x)))                
wid['mat']=wid['mat'].apply(lambda x: re.sub(r'((?<=\d)\s?cm)|((?<=\d)\s?(?="))',' cm',str(x))).apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)])).apply(lambda x: natsorted(x)).apply(lambda x: re.sub(r"\s?'\s?(?=\,)|(?<=\[)\s?'\s?|(?<=\,)\s?'\s?|\s?'\s?(?=\])",'"',str(x)))

1803


In [98]:
wid['Q:width']=wid['mat']
match_width=wid[['external_id','Q:width']]

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/Liverpool - -{get_df_name(matches)}-{today}-match_width.csv',index=False) 
looks_good('Liverpool', match_width,today)

In [94]:
wid_na=w[w['matches'].astype(str)=='[]']
print(len(wid_na))

5


In [96]:
wid_na['Q:width']='n/a'
match_width_wid_na=wid_na[['external_id','Q:width']]

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/Liverpool - -{get_df_name(matches)}-{today}-match_width_wid_na.csv',index=False) 
looks_good('Liverpool', match_width_wid_na,today)

In [24]:
pat='''(?i)((?:height(?:\s?product)|height of (?:the )?product\s?(?:bag\:?)?|product.?height\s?)\s?\d+(?:\.\d+)?\s?(?:cm)?)|(\d+\s?cm approximate height(?! of the))|()'''
h['matches']=h['long_desc'].apply(lambda x: re_extract(pat,str(x)))
hei=h[h['matches'].astype(str)!='[]']
print(len(hei))

hei['mat']=hei['matches'].apply(lambda x: re.sub(r'(?i)(Height of the product\s?)','',str(x)))

1766


In [35]:
hei['mat']=hei['matches'].apply(lambda x: re.sub(r'(?i)(Height of the product\s?)|(\s?Bag\s?\:)|(Product\s?height\s?)|(height\s?product\s?)|(height of product\s?)','',str(x))).apply(lambda x: re.sub(r"\s?'\s?(?=\,)|(?<=\[)\s?'\s?|(?<=\,)\s?'\s?|\s?'\s?(?=\])",'"',str(x))).apply(lambda x: re.sub(r'((?<=\d)cm)|((?<=\d)(?="))',' cm',str(x)))                               
hei['mat'].explode().value_counts()[0:500]

["17 cm"]       115
["24 cm"]       108
["25 cm"]       105
["23 cm"]        96
["30 cm"]        90
["26 cm"]        87
["15 cm"]        78
["29 cm"]        78
["14 cm"]        73
["16 cm"]        73
["20 cm"]        68
["13 cm"]        67
["22 cm"]        66
["19 cm"]        61
["18 cm"]        61
["27 cm"]        57
["32 cm"]        51
["28 cm"]        45
["12 cm"]        37
["21 cm"]        36
["33 cm"]        35
["31 cm"]        31
["35 cm"]        27
["40 cm"]        25
["38 cm"]        22
["34 cm"]        21
["11 cm"]        18
["36 cm"]        14
["17.5 cm"]       9
["43 cm"]         8
["14.5 cm"]       8
["37 cm"]         7
["10 cm"]         7
["25.5 cm"]       7
["39 cm"]         5
["27.5 cm"]       5
["10.5 cm"]       5
["42 cm"]         5
["45 cm"]         4
["15.5 cm"]       4
["12.7 cm"]       3
["22.5 cm"]       3
["20.5 cm"]       3
["44 cm"]         3
["16.5 cm"]       3
["10.2 cm"]       2
["20.9 cm"]       2
["14.9 cm"]       2
["75 cm"]         2
["13.5 cm"]       2


In [36]:
hei['Q:height']=hei['mat']
match_height=hei[['external_id','Q:height']]

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/Liverpool - -{get_df_name(matches)}-{today}-match_height.csv',index=False) 
looks_good('Liverpool', match_height,today)

In [26]:
hei_na=h[h['matches'].astype(str)=='[]']
print(len(hei_na))


hei_na['Q:height']='n/a'
match_height_na=hei_na[['external_id','Q:height']]

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/Liverpool - -{get_df_name(matches)}-{today}-match_height_na.csv',index=False) 
looks_good('Liverpool', match_height_na,today)

17


In [6]:
connection_inputs=df.loc[df['attribute'].astype(str)=='connection_inputs']

In [30]:
# connection_inputs['value'].explode().value_counts()

In [7]:
# connection inputs
pat='''(?i)((?:ethernet|HDMI|micro.?sd|thunderbolt|usb.?type.?(?:A|B|C)).{0,5}(?:.?port.?number.?)?.{0,30}(?:does not have|none|no).{0,10})|()'''
print('Connection Inputs')
connection_inputs, connection_inputss, na_connection_input, connection_inputs_custom, connection_inputs_not_custom=curation(connection_inputs,pat)

Connection Inputs
Total DF: 395
Total number of SKUs labeled N/A: 104
None Found: 1
Matches: 394
Custom Fields Only: 0
Not in Custom Fields: 394
##########################################################



In [32]:
# connection_inputs['m_long'].apply(lambda x: re.sub(r'','',str(x)))#['m_long'].explode().value_counts()[0:500]

In [8]:
x=connection_inputs[connection_inputs['m_long'].astype(str)!='[]']
print(len(x))

394


In [9]:
x['m']=x['m_long'].apply(lambda x: re.sub(r"\s?'\s?(?=\,)|(?<=\[)\s?'\s?|(?<=\,)\s?'\s?|\s?'\s?(?=\])",'"',str(x))).apply(lambda x: re.sub(r'''(?i)(\s?port.?number\s?)|((?<=does not have)[^"]+)|(ports?)|(\d+\.\d+)|(inputs?)''','',str(x))).apply(lambda x: re.sub(r'(\s{2,})','',str(x)))

In [10]:
x['m_long']=x['m_long'].apply(lambda x: re.sub(r"\s?'\s?(?=\,)|(?<=\[)\s?'\s?|(?<=\,)\s?'\s?|\s?'\s?(?=\])",'"',str(x))).apply(lambda x: re.sub(r"(?<=\d)\s(?=\.\d)",'',str(x))).apply(lambda x: re.sub(r'((?<![A-z])N o(?![A-z]))','No',str(x)))

In [11]:
pat='''(?i)((?:ethernet(?!\s(?:card|1\s?graphics card capacity|port|ports|port number))(?!(?:card|1\s?graphics card|port|ports|port number))|ethernet(?:\s?card|\s?1\s?graphics card capacity|\s?port(?!s)(?! number)|\s?ports|\s?port number )(?!\s(?:card|1\s?graphics card|port|ports|port number))(?!(?:card|1\s?graphics card|port|ports|port number))|HDMI|HDMI(?:\s?input(?!\s))|HDMI(?:\s?inputs)|micro.?sd|thunderbolt\s?ports(?! number)|Thunderbolt(?! port)|thunderbolt\s?port(?!s)(?! number)|thunderbolt ports number|thunderbolt\s?port(?!s)(?:\s?number\s?)|USB.?type?.?(?:A|B|C)\s?(?:(?:\d+\s?\.\d+\s?)?ports?.?(?:number)?\s?(?:\d+(?!\.)\s?)|\d+\s?\.\d+(?:\s?port.?(?:number)?)?)?(?:emmc storage capacity|hdd storage capacity)?(?!\s?(?:hdd|does not|None|no|not included|it does not have|card|input|port|number|s|\d|emmc)))(?:\s?ports?\s?(?:number)?)?(?!\s?(?:(?:it )?does not|None|no|not included|s|\d|\.|card|port|input|charger)))|()'''                              
x['match']=x['long_desc'].apply(lambda x: re_extract(pat,str(x)))
y=x[x['match'].astype(str)!='[]']
z=x[x['match'].astype(str)=='[]']

print(len(x))
print(len(y))
print(len(z))

394
290
104


In [12]:
y['matches']=y['match'].apply(lambda x: re.sub(r"\s?'\s?(?=\,)|(?<=\[)\s?'\s?|(?<=\,)\s?'\s?|\s?'\s?(?=\])",'"',str(x))).apply(lambda x: re.sub(r'(?i)(port number(?: \d+))|(\s?\d+\.\d+\s?)|(port\s?\d*)|(number)|(s\s?\d+)|()','',str(x))).apply(lambda x: re.sub(r'(?i)(\s?type[\-\s]?)',' Type ',str(x))).apply(lambda x: re.sub(r'(?i)(\s?Ethernet\s?card\s?)','Ethernet',str(x))).apply(lambda x: re.sub(r'(?i)(\s?(?<!U)s(?![A-z])\s?\d*)','',str(x))).apply(lambda x: re.sub(r'(ethernet)','Ethernet',str(x))).apply(lambda x: re.sub(r'(thunderbolt)','Thunderbolt',str(x))) .apply(lambda x: re.sub(r'(HDMI )','HDMI',str(x)))                    
y['matches']=y['matches'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)])).apply(lambda x: natsorted(x)).apply(lambda x: re.sub(r"\s?'\s?(?=\,)|(?<=\[)\s?'\s?|(?<=\,)\s?'\s?|\s?'\s?(?=\])",'"',str(x)))


In [13]:
y['matches'].explode().value_counts()

["USB Type A","USB Type C"]                 73
["USB Type A"]                              54
["USB Type C"]                              41
["USB Type B"]                              22
["Ethernet"]                                16
["Thunderbolt"]                             13
["Micro SD"]                                11
["Ethernet","USB Type A"]                    8
["Ethernet","USB Type C"]                    8
["Ethernet","USB Type B"]                    8
["Micro SD","USB Type A"]                    5
["USB Type A","USB Type B","USB Type C"]     4
["Ethernet","USB Type A","USB Type C"]       4
["USB Type B","USB Type C"]                  3
["Micro SD","USB Type B"]                    3
["Ethernet","Micro SD","USB Type B"]         3
["Micro SD","USB Type C"]                    2
["Ethernet","Micro SD","USB Type C"]         2
["HDMI","USB Type A","USB Type C"]           2
["USB Type A","USB Type B"]                  2
["Ethernet","Micro SD"]                      2
["Ethernet","

In [152]:
y['Q:connection_inputs']=y['matches']
match_connection_inputs=y[['external_id','Q:connection_inputs']]

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/Liverpool - -{get_df_name(matches)}-{today}-match_connection_inputs.csv',index=False) 
looks_good('Liverpool', match_connection_inputs,today)

In [153]:
z['Q:connection_inputs']='n/a'
match_connection_inputs_na=z[['external_id','Q:connection_inputs']]

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/Liverpool - -{get_df_name(matches)}-{today}-match_connection_inputs_na.csv',index=False) 
looks_good('Liverpool', match_connection_inputs_na,today)

In [5]:
bag_style=df.loc[df['attribute'].astype(str)=='bag_style']
capacity=df.loc[df['attribute'].astype(str)=='capacity']
care=df.loc[df['attribute'].astype(str)=='care']
case_color=df.loc[df['attribute'].astype(str)=='case_color']
case_shape=df.loc[df['attribute'].astype(str)=='case_shape']
case_size=df.loc[df['attribute'].astype(str)=='case_size']


case_size_range=df.loc[df['attribute'].astype(str)=='case_size_range']
closure=df.loc[df['attribute'].astype(str)=='closure']
color=df.loc[df['attribute'].astype(str)=='color']


compartment_count=df.loc[df['attribute'].astype(str)=='compartment_count']
connection_inputs=df.loc[df['attribute'].astype(str)=='connection_inputs']
depth=df.loc[df['attribute'].astype(str)=='depth']
features=df.loc[df['attribute'].astype(str)=='features']
footwear_size=df.loc[df['attribute'].astype(str)=='footwear_size']
gender=df.loc[df['attribute'].astype(str)=='gender']
graphics_card_capacity=df.loc[df['attribute'].astype(str)=='graphics_card_capacity']
graphics_card_type=df.loc[df['attribute'].astype(str)=='graphics_card_type']
hard_drive_capacity=df.loc[df['attribute'].astype(str)=='hard_drive_capacity']
hard_drive_type=df.loc[df['attribute'].astype(str)=='hard_drive_type']



heel_height=df.loc[df['attribute'].astype(str)=='heel_height']
material=df.loc[df['attribute'].astype(str)=='material']
maximum_charge_time=df.loc[df['attribute'].astype(str)=='maximum_charge_time']
processor=df.loc[df['attribute'].astype(str)=='processor']
processor_series=df.loc[df['attribute'].astype(str)=='processor_series']
product_type=df.loc[df['attribute'].astype(str)=='product_type']
random_access_memory_ram=df.loc[df['attribute'].astype(str)=='random_access_memory_ram']
screen_size=df.loc[df['attribute'].astype(str)=='screen_size']


toe_shape=df.loc[df['attribute'].astype(str)=='toe_shape']
watch_band_color=df.loc[df['attribute'].astype(str)=='watch_band_color']
watch_band_material=df.loc[df['attribute'].astype(str)=='watch_band_material']
watch_case_material=df.loc[df['attribute'].astype(str)=='watch_case_material']

watch_face_color=df.loc[df['attribute'].astype(str)=='watch_face_color']
watch_style=df.loc[df['attribute'].astype(str)=='watch_style']
weight=df.loc[df['attribute'].astype(str)=='weight']
width=df.loc[df['attribute'].astype(str)=='width']

water_resistance=df.loc[df['attribute'].astype(str)=='water_resistance']
height=df.loc[df['attribute'].astype(str)=='height']


In [54]:
print(len(capacity))

458


In [257]:
screen_size[screen_size['m_long'].astype(str)=='Product width 32 cm Number of USB type B 2.0 port']

Empty DataFrame
Columns: [attribute, buckets, bucket_id, value, customer_name, external_id, product_name, long_desc, curated_date, resolution, curation_tasks.curated_by, custom_fields, m_long, m_name, m_custom]
Index: []

In [291]:
width['value'].explode().value_counts()

30 cm                     142
8.66 in                   116
n/a                       113
11.81 in                  112
9.84 in                    98
24 cm                      96
13 cm                      86
23 cm                      86
14 cm                      85
15 cm                      84
10 cm                      82
22 cm                      77
25 cm                      77
32 cm                      77
4.72 in                    74
12 cm                      74
20 cm                      71
29 cm                      71
10.63 in                   71
10.24 in                   67
12.6 in                    66
26 cm                      65
9.06 in                    65
9.45 in                    63
35 cm                      62
27 cm                      61
11.42 in                   60
11 cm                      59
28 cm                      57
11.02 in                   56
13.78 in                   55
31 cm                      55
5.12 in                    48
8 cm      

In [288]:
# water resistance
pat='''(?i)(.{0,30}(?:not )?(?:water.?proof|water.?resistance|resistance.?into.?water|(?<!shock )resistance|resistance.?water|resistance.?to.?water|water.?proof|water.?resistance|resistance.?into.?water|resistance|resistance.?water|resistance.?to.?water).{0,30})|()'''
print('Water Resistance')
water_resistance, water_resistances, na_water_resistance, water_resistance_custom, water_resistance_not_custom=curation(water_resistance,pat)

Water Resistance
Total DF: 3802
Total number of SKUs labeled N/A: 172
None Found: 12
Matches: 3790
Custom Fields Only: 0
Not in Custom Fields: 3790
##########################################################



In [293]:
width['m_long'].explode().value_counts()[0:500]

oof No Compartment for Laptop 15 inches Capacity 1 liter Depth of the      7
 to protect your laptop up to 15.6 inches, Tablet or any device. In add    6
losure Compartment for Laptop 15.6 inches Capacity 16 liters Depth of t    4
m approximate Compartment for 15.6-inch tablet Includes Padlock No Ca      4
t width 31 cm Compartment for 16-inch tablet Includes padlock No Ty        3
 to protect your laptop up to 16 inches, Tablet or any device. Its la      3
asts, semi-soft construction, 2-in-1 function, folding flap that           3
Waterproof No Compartment for 16-inch laptop Family or collection W        3
 to protect your laptop up to 15.6 inches, Tablet or any device. Its st    3
ction Embossed with monogram. 2-in-1 handles for hand and should           3
erproof No Laptop compartment 15 inches Capacity 1 liter Product dept      2
2-in-1 bag, open leather bag and i                                         1
use of adapters. Protect your 15-inch MacBook or 14-inch PC in the         1

In [137]:
# x=capacity[(capacity['m_long'].astype(str)=='[]')&(capacity['m_name'].astype(str)=='[]')&(capacity['m_custom'].astype(str)=='[]')]
# print(len(x))

In [149]:
# bag style
pat='''(?i)(.{0,15}(?:back.?pack|bowler|bucket|clutch|cross.?body|duffel|hobo|lap.?top.?bag|messenger|pouch|saddle(?!.bag)(?!bag)|seat.?bag|satchel|shoulder.?bag|tote|wristle|bag.?style|Type of bag Shoulder).{0,15})|()'''
print('Bag Style')
bag_style,bag_styles, na_bag_style, bag_style_custom, bag_style_not_custom=curation(bag_style,pat)


# capacity
pat='''(?i)(.{0,50}(?:\d+(?:\.\d+)?[^\d]{0,5}L(?:iters?)?(?![A-z])|Liter|capacity).{0,30})|()'''
print('Capacity')
capacity, capacitys, na_capacity, capacity_custom, bag_style_not_custom=curation(capacity,pat)


# Care
pat='''(?i)(.{0,30}(?:hand.?wash|wash.{0,10}hand|wipe.?clean|clean|clean.?with.{0,5}semi.?damp.?cloth).{0,30})|()'''
print('Care')
care, cares, na_care, care_custom, care_not_custom=curation(care,pat)


# Case Color
pat='''(?i)(.{0,30}(?:black|blue|brown|clear|transparent|gold|grey|gray|green|multi.?color|orange|pink|purple|(?<![A-z])red(?![A-z])|rose.?gold|silver|(?<![A-z])tan(?![A-z])|white|yellow).{0,30})|()'''                           
print('Case Color')
case_color,case_colors, na_case_color, color_custom, color_not_custom=curation(case_color,pat)


# Shape
pat='''(?i)(.{0,30}(?:circle|square|triangle|rectangle|pentagon|hexagon|octagon|rhombus|trapezoid|parallelogram|kite|oval|ellipse|crescent|star|heart|diamond|rectangular|round|triangular).{0,30})|((?:sneaker|tennis.?shoe))|()'''
print('Shape')
case_shape,case_shapes, na_case_shape, case_shape_custom, case_shape_not_custom=curation(case_shape,pat)

In [158]:
# Case size
pat='''(?i)(.{0,30}case.?size.{0,20}(?:\d+(?:\.\d+)?.{0,5}(?:mm|millimeter)|specification|box.?size|spanish|caja|diameter|details of the (?:dial|face)|case.?size).{0,30})|(.{0,30}(?:\d+(?:\.\d+)?.{0,5}(?:mm|millimeter)|specification|box.?size|spanish|caja|diameter|details of the (?:dial|face)|case.?size).{0,20}case.?size)|(.{0,30case.{0,30}})|()'''
print('Case Size')
case_size, case_sizes, na_case_size, case_size_custom, case_size_not_custom=curation(case_size,pat)


# Case size range
pat='''(?i)(.{0,30}case.?size.{0,20}(?:\d+(?:\.\d+)?.{0,5}(?:mm|millimeter)|specification|box.?size|spanish|caja|diameter|details of the (?:dial|face)|case.?size).{0,30})|(.{0,30}(?:\d+(?:\.\d+)?.{0,5}(?:mm|millimeter)|specification|box.?size|spanish|caja|diameter|details of the (?:dial|face)|case.?size).{0,20}case.?size)|(.{0,30case.{0,30}})|()'''
print('Case Size Range')
case_size_range, case_size_ranges, na_case_size_range, case_size_range_custom, case_size_range_not_custom=curation(case_size_range,pat)


# closure
pat='''(?i)(.{0,30}(?:buckle|button|clasp|draw.?string|lace.?up|magnetic|slip.?on|snap|velcro|zipper).{0,30})|()'''
print('Closure')
closure, closures, na_closure, closure_custom, closure_not_custom=curation(closure,pat)


# color
pat='''(?i)(.{0,30}(?:assorted|multi.?color|rose.?gold|platinum|off.?white|cream|clear|screen.?protector|transparent|black|gray|grey|silver|gainsboro|white|snow|brown|coral|indian.?red|fire.?brick|maroon|(?<![A-z])red(?![A-z])|misty.?rose|salmon|orange|sienna|sea.?shell|chocolate|peach|peru|linen|bisque|burly.?wood|\btan\b|blanched.?almond|papaya.?whip|moccasin|wheat|oldlace|floral|golden.?rod|cornssilk|gold|lemon.?chiffon|khaki|ivory|beige|yellow|olive|green|charteuse|honey.?dew|lime|turquoise|azure|cyan|teal|aqua|blue|corn.?flower|lavender|navy|slate|purple|violet|indigo|dark.?orchid|thistle|plum|magenta|fuchsia|orchid|pink|crimson).{0,30})|()'''                           
print('Color')
color, colors, na_color, color_custom, color_not_custom=curation(color,pat)


# compartment count
pat='''(?i)(.{0,30}(?:number of compartments?.{0,5}\d+|(?:(?<!\.)(?<!\.)\d+|double|one|two|three|four|five|six|seven|eight|nine|ten)(?!in)(?! in)(?!mm)(?! mm)(?!cm)(?! cm)(?!\%)(?!\d)[^\.\d]{0,5}compartments?|compartments?[^\.]{0,3}(?:\d*\d|one|two|three|four|five|six|seven|eight|nine|ten)(?!in)(?! in)(?!mm)(?! mm)(?!cm)(?! cm)(?!\%)(?!\d)).{0,30})|()'''
print('Compartment Count')
compartment_count, compartment_counts, na_compartment_count, compartment_count_custom, compartment_count_not_custom=curation(compartment_count,pat)

In [167]:
# connection inputs
pat='''(?i)(.{0,30}(?:ethernet|HDMI|micro.?sd|thunderbolt|usb|type.?(?:A|B|C)|connection.?input).{0,30})|()'''
print('Connection Inputs')
connection_inputs, connection_inputss, na_connection_input, connection_inputs_custom, connection_inputs_not_custom=curation(connection_inputs,pat)


# Depth
pat='''(?i)(.{0,30}(?:depth|\d+(?:\.\d+)?.{0,10}(?:inc?h?e?s?|mm|milli|f(?:oo|ee)?t).{0,5}D(?:epths?)?(?![A-z])).{0,30})|()'''
print('Depth')
depth,depths, na_depth, depth_custom, depth_not_custom=curation(depth,pat)




In [ ]:
# features














In [191]:
# footwear size
pat='''(?i)(.{0,30}(?:\d+(?:\.\d+)?.{0,10}(?:cm|centi)|foot.?wear.?size).{0,30})|()'''
footwear_size, footwear_sizes, na_footwear_size, footwear_size_custom, footwear_size_not_custom=curation(footwear_size,pat)


# gender
pat='''(?i)(.{0,30}(?:gender|\bman\b|woman|boy|girl|\bmen\b|women|unisex|for women|female|(?<!fe)male|lady|unisex|boys and girls|girls and boys|men and women|women and men|\bman\b|woman|boy(?!friend)|girl(?!friend)(?! brand)|\bmen\b|women|unisex|for women|female|(?<!fe)male|lady).{0,30})|()'''
gender, genders, na_gender, gender_custom, gender_not_custom=curation(gender,pat)


# graphic card capacity
pat='''(?i)(.{0,30}(?:(?:10|12|14|16|18|1|20|22|24|2|32|4|6|8).{0,5}(?:giga|GB)|graphic card capacity).{0,30})|()'''
graphics_card_capacity, graphics_card_capacitys, na_graphics_card_capacity, graphics_card_capacity_custom,graphics_card_capacity_not_custom=curation(graphics_card_capacity,pat)

In [213]:
# graphic card type
pat='''(?i)(.{0,30}(?:AMD.?(?:radeon.{0,4}|vega.?\d+)|intel.?(?:iris.{0,3}|uhd)|nvidia.?(?:geforce.{0,4}|geoforce.?quadro)|graphic.?card).{0,30})|()'''
print('Graphic Card Type')
graphics_card_type, graphics_card_types, na_graphics_card_type, graphics_card_type_custom, graphics_card_type_not_custom=curation(graphics_card_type,pat)


# hard drive capacity
pat='''(?i)(.{0,30}(?:hard.?drive|(?:128|256|500|512|64|10\d\d).{0,5}(?:GB|giga)|(?:1).{0,5}(?:TB|tera)|hard.?drive.?capacity).{0,30})|()'''
print('Hard Drive Capacity')
hard_drive_capacity, hard_drive_capacitys, na_hard_drive_capacity, hard_drive_capacity_custom, hard_drive_capacity_not_custom=curation(hard_drive_capacity,pat)


# hard drive type
pat='''(?i)(.{0,30}(?:emmc|hhd|ssd|hard.?drive.?type|solid.?storage|hard drive|hard disk drive|solid.?state|Embedded.?Multi.?Media).{0,30})|()'''
print('Hard Drive Type')
hard_drive_type, hard_drive_types, na_hard_drive_type, hard_drive_type_custom, hard_drive_type_not_custom=curation(hard_drive_type,pat)


# heel height
pat='''(?i)(.{0,30}(?:heel(?:.?height(?:\/platform)|\/platform(?:.?height))?)\s?\d+(?:\.\d+)?\s?(?:cm|mm|centi|mili|in|f(?:oo|eet)?t).{0,30})|(.{0,30}(?:without.?(?:heel|platform)|(?:no.?heel|flat|no.?platform)).{0,30})|(.{0,30}platform.?type.{0,30})|()'''
print('Heel Height')
heel_height, heel_heights, na_heel_height, heel_height_custom, heel_height_not_custom=curation(heel_height,pat)




In [217]:
# # height
# pat='''(?i)((?:(?<!heel )(?<!hell)(?<!platform )(?<!platform)height(?! heel)|\d+(?:\.\d+)?.{0,5}(?:inc?h?e?s?|f(?:oo|ee)?t|mm|milli|cm|centi)(?! heel)[^[A-z]]{0,5}H(?:eight)?))|()'''
# print('Height')
# height, heights, na_height, height_custom, height_not_custom=curation(height,pat)

In [236]:

# material
pat='''(?i)(.{0,30}(?:material|cotton|fabric|textile|faux.?leather|Curpiel|foam|leather|memory.?foam|nylon|plastic|polyester|polyurethane|recycled.?material|rubber|satin|silk|suede|synthetic|\btpr\b|Thermoplastic.?Rubber|vinyl|Vinipeil|Vinipiel).{0,30})|()'''
print('Material')
material, materials, na_material, material_custom, material_not_custom=curation(material,pat)


# maximum charge time
pat='''(?i)(.{0,30}(?:charge.?time|\d+.{0,3}hour).{0,30})|()'''
print('Maximum Charge Time')
maximum_charge_time, maximum_charge_times, na_maximum_charge_time, maximum_charge_time_custom, maximum_charge_time_not_custom=curation(maximum_charge_time,pat)


# # processor type should be the same call out as series
# pat='''(?i)(.{0,30}(?:amd|apple|intel).{0,30})|()'''
# print('Processor')
# processor, processors, na_processor, processor_custom, processor_not_custom=curation(processor,pat)


# processor series
pat='''(?i)(.{0,30}(?:amd.?(?:a.?series|athalon|e.?series|ryzen|threadripper)|apple.?(?:m\d)|intel.?(?:celeron|core.?i?\d+|i\d+|pentium|xeon)).{0,30})|()'''
print('Processor Series')
processor_series, processor_seriess, na_processor_series, processor_series_custom, processor_series_not_custom=curation(processor_series,pat)

In [109]:
# product Type













In [270]:
# ram
pat='''(?i)(.{0,40}(?:Ram memory capacity|memory|(?<!graphics card capacity )(?<!\d)(?:12|16|32|4|8)[^\d]{0,3}GB|(?:12\d\d\d|16\d\d\d|32\d\d\d|4\d\d\d|8\d\d\d)[^\d]{0,3}MB).{0,40})|()'''
print('Randome Access Memory')
random_access_memory_ram, random_access_memory_rams, na_random_access_memory_ram, random_access_memory_ram_custom, random_access_memory_ram_not_custom=curation(random_access_memory_ram,pat)


# screen size
pat='''(?i)(.{0,30}(?:screen|\d+(?:\.\d+)?.?(?:in|mm|cm)).{0,30})|()'''
print('Screen Size')
screen_size, screen_sizes, na_screen_size, screen_size_custom, screen_size_not_custom=curation(screen_size,pat)


# toe shape
pat='''(?i)(.{0,30}(?:toe.?shape|almond|cap|closed|moc|peep.?toe|pointed|round|shell|square).{0,30})|()'''
print('Toe Shape')
toe_shape, toe_shapes, na_toe_shape, toe_shape_custom, toe_shape_not_custom=curation(toe_shape,pat)


# # watch band color
# pat='''(?i)(.{0,30}(?:assorted|multi.?color|rose.?gold|platinum|off.?white|cream|clear|screen.?protector|transparent|black|gray|grey|silver|gainsboro|white|snow|brown|coral|indian.?red|fire.?brick|maroon|red|misty.?rose|salmon|orange|sienna|sea.?shell|chocolate|peach|peru|linen|bisque|burly.?wood|\btan\b|blanched.?almond|papaya.?whip|moccasin|wheat|oldlace|floral|golden.?rod|cornssilk|gold|lemon.?chiffon|khaki|ivory|beige|yellow|olive|green|charteuse|honey.?dew|lime|turquoise|azure|cyan|teal|aqua|blue|corn.?flower|lavender|navy|slate|purple|violet|indigo|dark.?orchid|thistle|plum|magenta|fuchsia|orchid|pink|crimson).{0,30})|()'''                           
# print('Watch Band Color')
# watch_band_color, watch_band_colors, na_watch_band_color, watch_band_color_custom, watch_band_color_not_custom=curation(watch_band_color,pat)



# # watch band material
# pat='''(?i)(.{0,30}(?:watch.?band.?material|ceramic|fabric|textile|faux.?leather|curpiel|(?:faux.)(?<!faux)leather|metal|neoprene|nylon|plastic|silicone|stainless.?steel|(?<!stainless.)(?<!stainless)steel|synthetic).{0,30})|()'''
# print('Watch Band Material')
# watch_band_material, watch_band_materials, na_watch_band_material, watch_band_material_custom, watch_band_material_not_custom=curation(watch_band_material,pat)

In [269]:
# # watch case material
# pat='''(?i)(.{0,30}(?:ceramic|glass|gold|metal|nylon|plastic|polycarbonate|polyurethane|resin|silver|platinum|stainless.?steel|(?<!stainless.)(?<!stainless)steel).{0,30})|()'''
# print('Watch Case Material')
# watch_case_material, watch_case_materials, na_watch_case_material,watch_case_material_custom, watch_case_material_not_custom=curation(watch_case_material,pat)


# # watch face color
# pat='''(?i)(.{0,30}(?:assorted|multi.?color|rose.?gold|platinum|off.?white|cream|clear|screen.?protector|transparent|black|gray|grey|silver|gainsboro|white|snow|brown|coral|indian.?red|fire.?brick|maroon|red|misty.?rose|salmon|orange|sienna|sea.?shell|chocolate|peach|peru|linen|bisque|burly.?wood|\btan\b|blanched.?almond|papaya.?whip|moccasin|wheat|oldlace|floral|golden.?rod|cornssilk|gold|lemon.?chiffon|khaki|ivory|beige|yellow|olive|green|charteuse|honey.?dew|lime|turquoise|azure|cyan|teal|aqua|blue|corn.?flower|lavender|navy|slate|purple|violet|indigo|dark.?orchid|thistle|plum|magenta|fuchsia|orchid|pink|crimson).{0,30})|()'''                           
# print('Watch Face Color')
# watch_face_color, watch_face_colors, na_watch_face_color, watch_face_color_custom, watch_face_color_not_custom=curation(watch_face_color,pat)


# watch style
pat='''(?i)(.{0,30}(?:bangle|bracelet|casual|fabric|nylon|silicone|chronograph|mechanism|dress|platinum|silver|rose.?gold|(?:yellow|white).?gold|leather.?band|fashion|luxury|fine.?watch|alta.?reloj|rolex|cartier|gucci|versace|pocket.?watch|sports?.?watch|fitness.?track|performance.?watch|pedometer|heart.?rate|(?:silicon|plastic).?band).{0,30})|()'''
print('Watch Style')
watch_style, watch_styles, na_watch_style, watch_style_custom, watch_style_not_custom=curation(watch_style,pat)

In [112]:
# water resistance
pat='''(?i)(.{0,30}(?:water.?proof|water.?resistance|resistance.?into.?water|resistance|resistance.?water|resistance.?to.?water|water.?proof|water.?resistance|resistance.?into.?water|resistance|resistance.?water|resistance.?to.?water).{0,30})|(.{0,30}(?:water.?proof|water.?resistance|resistance.?into.?water|resistance|resistance.?water|resistance.?to.?water)[^\d]{0,30})|(.{0,30}(?:\d+\s?m(?:eters?)?(?![A-z])(?![0-9])|\d+(?:\.\d+)?[^\d[A-z]]?M(?:eters?)?(?![A-z])(?![0-9])).{0,30})|()'''
print('Water Resistance')
water_resistance, water_resistances, na_water_resistance, water_resistance_custom, water_resistance_not_custom=curation(water_resistance,pat)

Water Resistance
Total DF: 3802
 
Total number of SKUs labeled N/A: 172
 
None Found: 7

Matches: 3795

Custom Fields Only: 26

Not in Custom Fields: 3769

##########################################################



In [113]:
# weight
pat='''(?i)(.{0,30}(?:\d+(?:\.\d+)?.?g(?:rams?)?(?![A-z])).{0,30})|()'''
print('Weight')
weight, weights, na_weight, weight_custom, weight_not_custom=curation(weight,pat)


# width
pat='''(?i)(.{0,30}(?:\d+(?:\.\d+)?.?(?:inc?h?e?s?|f(?:oo|ee)?t)s?(?![A-z])).{0,30})|()'''
print('Width')
width, widths, na_width, width_custom, width_not_custom=curation(width,pat)

Weight
Total DF: 457
 
Total number of SKUs labeled N/A: 407
 
None Found: 417

Matches: 40

Custom Fields Only: 0

Not in Custom Fields: 40

##########################################################

Width
Total DF: 4096
 
Total number of SKUs labeled N/A: 113
 
None Found: 4066

Matches: 30

Custom Fields Only: 1

Not in Custom Fields: 29

##########################################################

